In [48]:
import pymongo
import json
from pprint import pprint

In [49]:
try :
    client = pymongo.MongoClient('mongodb://localhost:27017/')

    db = client.db_agg
    db.transactions.drop()

    transactions = db.transactions
except :
    print("connexion error")

try :
    with open('json/transactions.json') as json_data:
        data = json.load(json_data)
except :
    print("impossible d'ouvrir les fichiers json")

try:
    db.transactions.insert_many(data)
except:
    print("Insert many error")

In [50]:
# Q-1. Calculer le montant total des paiements ?
# # Indice : key : Payment.Total
match = {"$match":{"Payment":{"$ne": None}}}
group = {"$group" : {"_id" : None, "Total Payment" : { "$sum" : "$Payment.Total" }}}

for trans in transactions.aggregate( [match, group] ):
    pprint(trans)

{'Total Payment': 1230, '_id': None}


In [47]:
# Q-2. Calculer le montant total par transactions (montant total du panier) ?
unwind = { "$unwind" : '$Panier' }
group = {"$group" : {"_id" : "$TransactionId", "Total Panier" : { "$sum" : "$Panier.price" }}}

for trans in transactions.aggregate( [unwind, group] ):
    pprint(trans)

{'Total Panier': 410, '_id': 'tran4'}
{'Total Panier': 410, '_id': 'tran3'}
{'Total Panier': 410, '_id': 'tran2'}
{'Total Panier': 410, '_id': 'tran1'}


In [34]:
# Q-3. Calculate total payments (Payment.Total) for each payment type (Payment.Type) ?
match = {"$match":{"Payment":{"$ne": None}}}
group = {"$group" : {"_id" : "$Payment.Type", "Total Payment" : { "$sum" : "$Payment.Total" }}}

for trans in transactions.aggregate( [match, group] ):
    pprint(trans)

{'Total Payment': 410, '_id': 'Debit-Card'}
{'Total Payment': 820, '_id': 'Credit-Card'}


In [40]:
# Q-4. Trouver l'Id le plus élevé.
group = {"$group": { "_id" : None , "Id": {"$max": "$Id"} }}
sort = {"$sort" : {"max" : -1}}

for trans in transactions.aggregate([group, sort]):
    pprint(trans)

{'Id': 103, '_id': None}


In [32]:
# Q-5. Find the max price (Transaction.price) ?
unwind = { "$unwind" : '$Panier' }
group = { "$group" : {"_id" : None , "max_price" : { "$max" : "$Panier.price" }} }
sort = {"$sort" : {"max" : -1}}

for trans in transactions.aggregate( [unwind, group, sort] ):
    pprint(trans)

{'_id': None, 'max_price': 210}
